# NanoGPT

This is a jupyter notebook which followed the video tutotial from Andrej Karpathy in this [video](https://www.youtube.com/watch?v=kCc8FmEb1nY)

There are 2 repositories related to this repo:
1. [ng-video-lecture](https://github.com/karpathy/ng-video-lecture)
2. [nanoGPT](https://github.com/karpathy/nanoGPT)

### 1. Exploratory Data 

We take a look at the data in the `input.txt` file provided in the nanoGPT repo

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### 2. Tokenization and train/val split

we use a simple enumerate tokenizer and create the encode and decode lambda functions accordingly

In [5]:
# Create mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]     #encoder: take a string, output a list tob integers
decode = lambda l: ''.join([itos[i] for i in l])    #decoder: take a list of integers, output a string

print(encode("Hello there!"))
print(decode(encode("Hello there!")))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]
Hello there!


In [6]:
# We will encode the entire text dataset into a torch.tensor
import torch
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Train and validation split with 90 percent of the data into train and rest in validation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### 3. Data Loader: batches of chunks of data

We will convert the data in randomly sampled batches of block size 8. This means that we get a batch of 8 tokens at a time meaning 8 characters. Essentially the next character should be predicted.

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[block_size:]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When input is", context, "the target: ", target)

When input is tensor([18]) the target:  tensor(58)
When input is tensor([18, 47]) the target:  tensor(47)
When input is tensor([18, 47, 56]) the target:  tensor(64)
When input is tensor([18, 47, 56, 57]) the target:  tensor(43)
When input is tensor([18, 47, 56, 57, 58]) the target:  tensor(52)
When input is tensor([18, 47, 56, 57, 58,  1]) the target:  tensor(10)
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target:  tensor(0)
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target:  tensor(14)


In [10]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel
block_size = 8 # What is the maximum context length for predictions

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))   # Select 4 (batch_size) random locations from 0 to length of data - block_size (8)
    x = torch.stack([data[i:i+block_size] for i in ix]) # Some block
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # Offset by 1
    
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("____")

for b in range(batch_size): #batch dimension
    print("\n")
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print("When input is ", context.tolist(), " the target: ", target)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
____


When input is  [24]  the target:  tensor(43)
When input is  [24, 43]  the target:  tensor(58)
When input is  [24, 43, 58]  the target:  tensor(5)
When input is  [24, 43, 58, 5]  the target:  tensor(57)
When input is  [24, 43, 58, 5, 57]  the target:  tensor(1)
When input is  [24, 43, 58, 5, 57, 1]  the target:  tensor(46)
When input is  [24, 43, 58, 5, 57, 1, 46]  the target:  tensor(43)
When input is  [24, 43, 58, 5, 57, 1, 46, 43]  the target:  tensor(39)


When input is  [44]  the target:  tensor(53)
When input is  [44, 53]  the target:  tensor(56)
When input is  [44, 53, 56]  the target:  

In [11]:
print(xb)   # The input to our transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


### 4. Baseline: Bigram Language Model, Loss, Generation

The Bigram Language Model is one of the most easiest to implement and is covered in detail in this Andrej's Makemore series

In [12]:
import torch
import torch.nn as nn

from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integers
        # idx is the current context of some characters in a batch
        logits = self.token_embedding_table(idx)    # (B, T, C)     B = Batch(4), T = Time(8), C = Channel(vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # This function is supposed to take (B, T) and make it (B, T+1) -> (B, T+2) -> ... ->(B, T+max_new_tokens)

        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]   # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  #(B, 1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    
m = BigramLanguageModel(vocab_size)     #Vocab_size defined way back in cell 4
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


We expect the loss to be around -ln(1/65) = 4.1743872699

Meaning the initial prediction is not super-diffused and we are getting some entropy

Thus, the generate function was added to the class

In [13]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


Essentially what we did is give it a tensor of shape (1,1) containing 0 which we feed to the generate function.

### 5. Training the bigram model

In [14]:
# Create the optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32

for steps in range(10000):

    # sample a batch of data:
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.382369041442871


In [16]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


It seems to make more sense now. Andrej has provided a file named `bigram.py` in ng-video-lecture repo which has additional methods and optimizations. It also runs everything on a cuda device

### 6. Averaging past context with for loops, the weakest form of aggregation

#### The mathematical trick in self-attention

Every token should be defined by it's history and not the next token which is basically the future.



In [17]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2   # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We eant x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]    # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [20]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [21]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a@b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
--


Instead we use a lower traingular matrix

In [22]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a@b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
--


Using what we have learnt we try to vectorize

In [23]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C)    -->    (B, T, T) @ (B, T, C) ==> (B, T, C)
torch.allclose(xbow, xbow2)

True

Pytorch sees that wei is a (T, T) matrix so it creates the dimension B for wei and every (T, T) matrix is multiplied to the (T, C) batch

In [24]:
# version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

#### Basic self-attention mechanism

In [25]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32  #batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)    # (B, T, 16)
wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [26]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)